In [2]:
import re
import os
import csv
import pickle
import gensim
import sqlite3
import logging
import warnings
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
from tqdm import tqdm
from sklearn import svm
from pprint import pprint
from sklearn import metrics
from datetime import datetime
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import gensim.corpora as corpora
from nltk.corpus import stopwords
from sklearn.externals import joblib
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from nltk.stem.snowball import SnowballStemmer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import ldamulticore, CoherenceModel
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score,precision_score,recall_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer



%autosave 120
warnings.filterwarnings("ignore")

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Autosaving every 120 seconds


In [7]:
dataframe = pd.read_csv("/content/drive/My Drive/ML/data_with_all_tags.csv")
dataframe.shape

(13757, 81)

In [8]:
data = dataframe[['title', 'plot_synopsis', 'tags', 'split', 'CleanedSynopsis']]

data.shape

(13757, 5)

In [9]:
data["synopsis_words"] = data["CleanedSynopsis"].apply(lambda x: x.split())
data.head()

,title,plot_synopsis,tags,split,CleanedSynopsis,synopsis_words
0,$,"Set in Hamburg, West Germany, several criminal...",murder,test,set hamburg west germani sever crimin take adv...,"[set, hamburg, west, germani, sever, crimin, t..."
1,$windle,A 6th grader named Griffin Bing decides to gat...,flashback,train,grader name griffin bing decid gather entir gr...,"[grader, name, griffin, bing, decid, gather, e..."
2,'71,"Gary Hook, a new recruit to the British Army, ...","suspenseful, neo noir, murder, violence",train,gari hook new recruit british armi take leav m...,"[gari, hook, new, recruit, british, armi, take..."
3,'A' gai wak,Sergeant Dragon Ma (Jackie Chan) is part of th...,"cult, violence",train,sergeant dragon jacki chan part hong kong mari...,"[sergeant, dragon, jacki, chan, part, hong, ko..."
4,'Breaker' Morant,"In Pretoria, South Africa, in 1902, Major Char...","murder, anti war, violence, flashback, tragedy...",train,pretoria south africa major charl bolton rod m...,"[pretoria, south, africa, major, charl, bolton..."


In [0]:
data_words=[]
for sent in data["synopsis_words"].values:
    data_words.append(sent)

In [0]:
#Reference: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
start = datetime.now()

id2word = corpora.Dictionary(data_words)

corpus = [id2word.doc2bow(text) for text in data_words]

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           chunksize=10,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=100,
                                           per_word_topics=True,
                                           workers=7)

pprint(lda_model.print_topics())
print("Time taken to run this cell :", datetime.now() - start)

[(0,
  '0.009*"kill" + 0.006*"father" + 0.006*"get" + 0.005*"love" + 0.004*"famili" '
  '+ 0.004*"brother" + 0.004*"son" + 0.004*"meet" + 0.004*"take" + '
  '0.004*"fight"'),
 (1,
  '0.014*"tell" + 0.012*"go" + 0.011*"see" + 0.011*"say" + 0.011*"back" + '
  '0.010*"get" + 0.007*"ask" + 0.007*"find" + 0.007*"look" + 0.007*"room"'),
 (2,
  '0.015*"kill" + 0.011*"polic" + 0.007*"car" + 0.006*"shoot" + 0.006*"offic" '
  '+ 0.006*"frank" + 0.006*"man" + 0.006*"john" + 0.006*"find" + 0.005*"gun"'),
 (3,
  '0.008*"georg" + 0.008*"get" + 0.008*"billi" + 0.007*"jim" + 0.007*"ray" + '
  '0.006*"scott" + 0.006*"go" + 0.005*"rachel" + 0.005*"find" + 0.005*"bella"'),
 (4,
  '0.009*"hous" + 0.008*"find" + 0.007*"mother" + 0.007*"mari" + '
  '0.007*"father" + 0.006*"child" + 0.006*"kill" + 0.006*"home" + '
  '0.005*"woman" + 0.005*"famili"'),
 (5,
  '0.016*"go" + 0.015*"tell" + 0.014*"get" + 0.012*"say" + 0.012*"sam" + '
  '0.011*"david" + 0.009*"mike" + 0.009*"charli" + 0.008*"ask" + '
  '0.008*"sar

In [0]:
#Reference: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

start = datetime.now()

data_list = dataframe.CleanedSynopsis.values.tolist()

def format_topics_sentences(ldamodel=None, corpus=corpus, texts = data_list):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_words)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

print("Time taken to run this cell :", datetime.now() - start)

Time taken to run this cell : 0:04:16.213515


In [0]:
df_dominant_topic.head(5)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,2.0,0.6641,"kill, polic, car, shoot, offic, frank, man, jo...","[set, hamburg, west, germani, sever, crimin, t..."
1,1,5.0,0.2997,"go, tell, get, say, sam, david, mike, charli, ...","[grader, name, griffin, bing, decid, gather, e..."
2,2,8.0,0.5935,"kill, war, attack, soldier, ship, order, forc,...","[gari, hook, new, recruit, british, armi, take..."
3,3,8.0,0.4186,"kill, war, attack, soldier, ship, order, forc,...","[sergeant, dragon, jacki, chan, part, hong, ko..."
4,4,8.0,0.5042,"kill, war, attack, soldier, ship, order, forc,...","[pretoria, south, africa, major, charl, bolton..."


In [0]:
#Save the top topics in a CSV file.
df_dominant_topic.to_csv("dominant_topics.csv")

In [ ]:
#Load the topics data
df_topic = pd.read_csv("/content/drive/My Drive/ML/dominant_topics.csv")

#Combine the processed db with topics db
combined_df = pd.concat([data, df_topic], axis=1)

In [12]:
#Create a dataset for train and test
data_test=combined_df.loc[(combined_df['split'] == 'test')]
data_train=combined_df.loc[(combined_df['split'] == 'val') | (combined_df['split'] == 'train')]

#Split the whole plot data into train and test set
X_train_plots = data_train['CleanedSynopsis']
X_test_plots = data_test['CleanedSynopsis']

#Split the whole topics data into train and test set
X_train_topics = data_train['Keywords']
X_test_topics = data_test['Keywords']

#Split the tags
y_train = data_train['tags']
y_test = data_test['tags']

print("Number of points in training data: ",data_train.shape[0])
print("Number of points in test data: ",data_test.shape[0])

Number of points in training data:  10989
Number of points in test data:  2768


In [0]:
#Importing & Initializing the "CountVectorizer" object, which is scikit-learn's bag of words tool. By default 'split()' will tokenize each tag using space.
def tokenize(x):
    x=x.split(',')
    tags=[i.strip() for i in x] #Some tags contains whitespaces before them, so we need to strip them
    return tags

cnt_vectorizer = CountVectorizer(tokenizer = tokenize, binary='true').fit(y_train)
y_train_multilabel = cnt_vectorizer.transform(y_train)
y_test_multilabel = cnt_vectorizer.transform(y_test)

<h1> 1. TfidfVectorizer with 1 grams: </h1>

In [14]:
start = datetime.now()


vectorizer = TfidfVectorizer(min_df = 0.000009, max_features =250000,   tokenizer = lambda x: x.split(" "), ngram_range=(1, 1))
X_train_plots_multilabel = vectorizer.fit_transform(X_train_plots)
X_test_plots_multilabel = vectorizer.transform(X_test_plots)

#Use tf-idf vectorizer to vectorize the movie topic synopsis
vectorizer = TfidfVectorizer(min_df = 0.000009, max_features =250000,   tokenizer = lambda x: x.split(" "),  ngram_range=(1, 1))
X_train_topics_multilabel = vectorizer.fit_transform(X_train_topics)
X_test_topics_multilabel = vectorizer.transform(X_test_topics)

#Combine the sparse matrices into a single sparse matrix
from scipy.sparse import hstack
X_train_multilabel=hstack([X_train_plots_multilabel,X_train_topics_multilabel])
X_test_multilabel=hstack([X_test_plots_multilabel,X_test_topics_multilabel])

print("Time taken to run this cell :", datetime.now() - start)

Time taken to run this cell : 0:00:02.923173


In [15]:
print("Dimensions of train data X:",X_train_multilabel.shape, "Y :",y_train_multilabel.shape)
print("Dimensions of test data X:",X_test_multilabel.shape,"Y:",y_test_multilabel.shape)

Dimensions of train data X: (10989, 79067) Y : (10989, 71)
Dimensions of test data X: (2768, 79067) Y: (2768, 71)


<h2> 1.1 OneVsRestClassifier + MultinomialNB:</h2>

In [16]:
mb = MultinomialNB(class_prior = [0.5, 0.5])

clf = OneVsRestClassifier(mb)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=MultinomialNB(alpha=1.0, class_prior=[0.5, 0.5],
                                            fit_prior=True),
                    n_jobs=None)

In [17]:
prediction1 = clf.predict(X_test_multilabel)

precision1 = precision_score(y_test_multilabel, prediction1, average='micro')

recall1 = recall_score(y_test_multilabel, prediction1, average='micro')

f1_score1 = 2*((precision1 * recall1)/(precision1 + recall1))

print("precision1: {:.4f}, recall1: {:.4f}, F1-measure: {:.4f}".format(precision1, recall1, f1_score1))

precision1: 0.6576, recall1: 0.0538, F1-measure: 0.0994


for i in range(5):
    k = data_test.sample(1).index[0]
    print("Movie: ", data_test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction1[k])[0],"\n")

<h2> 1.2 OneVsRestClassifier + SGDClassifier with LOG Loss:</h2>

In [18]:
sgl = SGDClassifier(loss='log', class_weight='balanced')

clf = OneVsRestClassifier(sgl)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal', loss='log',
                                            max_iter=1000, n_iter_no_change=5,
                                            n_jobs=None, penalty='l2',
                                            power_t=0.5, random_state=None,
                                            shuffle=True, tol=0.001,
                                            validation_fraction=0.1, verbose=0,
                                            warm_start=False),
                    n_jobs=None)

In [19]:
prediction2 = clf.predict(X_test_multilabel)

precision2 = precision_score(y_test_multilabel, prediction2, average='micro')

recall2 = recall_score(y_test_multilabel, prediction2, average='micro')

f1_score2 = 2*((precision2 * recall2)/(precision2 + recall2))

print("precision2: {:.4f}, recall2: {:.4f}, F1-measure: {:.4f}".format(precision2, recall2, f1_score2))

precision2: 0.2735, recall2: 0.4574, F1-measure: 0.3423


for i in range(5):
    k = data_test.sample(1).index[0]
    print("Movie: ", data_test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction2[k])[0],"\n")

<h2> 1.3 OneVsRestClassifier + SGDClassifier with Hinge Loss:</h2>

In [20]:
sgh = SGDClassifier(loss='hinge', class_weight='balanced')

clf = OneVsRestClassifier(sgh)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal',
                                            loss='hinge', max_iter=1000,
                                            n_iter_no_change=5, n_jobs=None,
                                            penalty='l2', power_t=0.5,
                                            random_state=None, shuffle=True,
                                            tol=0.001, validation_fraction=0.1,
                                            verbose=0, warm_start=False),
                    n_jobs=None)

In [21]:
prediction3 = clf.predict(X_test_multilabel)

precision3 = precision_score(y_test_multilabel, prediction3, average='micro')

recall3 = recall_score(y_test_multilabel, prediction3, average='micro')

f1_score3 = 2*((precision3 * recall3)/(precision3 + recall3))

print("precision3: {:.4f}, recall3: {:.4f}, F1-measure: {:.4f}".format(precision3, recall3, f1_score3))

precision3: 0.2997, recall3: 0.3835, F1-measure: 0.3364


for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction3[k])[0],"\n")

<h2> 1.4  OneVsRestClassifier + LogisticRegression:</h2>

In [22]:
lr = LogisticRegression(class_weight='balanced')

clf = OneVsRestClassifier(lr)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight='balanced',
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [23]:
prediction4 = clf.predict(X_test_multilabel)

precision4 = precision_score(y_test_multilabel, prediction4, average='micro')

recall4 = recall_score(y_test_multilabel, prediction4, average='micro')

f1_score4 = 2*((precision4 * recall4)/(precision4 + recall4))

print("precision4: {:.4f}, recall4: {:.4f}, F1-measure: {:.4f}".format(precision4, recall4, f1_score4))

precision4: 0.3019, recall4: 0.4446, F1-measure: 0.3596


for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction4[k])[0],"\n")

<h1>2. TfidfVectorizer with (1 - 2 Grams):</h1>

In [24]:
start = datetime.now()

#Use tf-idf vectorizer to vectorize the movie topic synopsis
vectorizer = TfidfVectorizer(min_df = 0.000009, max_features =250000,   tokenizer = lambda x: x.split(" "), ngram_range=(1, 2))
X_train_plots_multilabel = vectorizer.fit_transform(X_train_plots)
X_test_plots_multilabel = vectorizer.transform(X_test_plots)

#Use tf-idf vectorizer to vectorize the movie topic synopsis
vectorizer = TfidfVectorizer(min_df = 0.000009, max_features =250000,  tokenizer = lambda x: x.split(" "),  ngram_range=(1, 2))
X_train_topics_multilabel = vectorizer.fit_transform(X_train_topics)
X_test_topics_multilabel = vectorizer.transform(X_test_topics)

#Combine the sparse matrices into a single sparse matrix
from scipy.sparse import hstack
X_train_multilabel=hstack([X_train_plots_multilabel,X_train_topics_multilabel])
X_test_multilabel=hstack([X_test_plots_multilabel,X_test_topics_multilabel])

print("Time taken to run this cell :", datetime.now() - start)

Time taken to run this cell : 0:00:25.287841


In [25]:
print("Dimensions of train data X:",X_train_multilabel.shape, "Y :",y_train_multilabel.shape)
print("Dimensions of test data X:",X_test_multilabel.shape,"Y:",y_test_multilabel.shape)

Dimensions of train data X: (10989, 250167) Y : (10989, 71)
Dimensions of test data X: (2768, 250167) Y: (2768, 71)


<H2> 2.1 OneVsRestClassifier + MultinomialNB :</H2>

In [26]:
mb = MultinomialNB(class_prior = [0.5, 0.5])

clf = OneVsRestClassifier(mb)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=MultinomialNB(alpha=1.0, class_prior=[0.5, 0.5],
                                            fit_prior=True),
                    n_jobs=None)

In [27]:
prediction5 = clf.predict(X_test_multilabel)

precision5 = precision_score(y_test_multilabel, prediction5, average='micro')

recall5 = recall_score(y_test_multilabel, prediction5, average='micro')

f1_score5 = 2*((precision5 * recall5)/(precision5 + recall5))

print("precision5: {:.4f}, recall5: {:.4f}, F1-measure: {:.4f}".format(precision5, recall5, f1_score5))

precision5: 0.6786, recall5: 0.0441, F1-measure: 0.0828


for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction5[k])[0],"\n")

<h2> 2.2 OneVsRestClassifier + SGDClassifier with LOG Loss :</h2>

In [28]:
sgl = SGDClassifier(loss='log', class_weight='balanced')

clf = OneVsRestClassifier(sgl)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal', loss='log',
                                            max_iter=1000, n_iter_no_change=5,
                                            n_jobs=None, penalty='l2',
                                            power_t=0.5, random_state=None,
                                            shuffle=True, tol=0.001,
                                            validation_fraction=0.1, verbose=0,
                                            warm_start=False),
                    n_jobs=None)

In [29]:
prediction6 = clf.predict(X_test_multilabel)

precision6 = precision_score(y_test_multilabel, prediction6, average='micro')

recall6 = recall_score(y_test_multilabel, prediction6, average='micro')

f1_score6 = 2*((precision6 * recall6)/(precision6 + recall6))

print("precision6: {:.4f}, recall6: {:.4f}, F1-measure: {:.4f}".format(precision6, recall6, f1_score6))

precision6: 0.2953, recall6: 0.4488, F1-measure: 0.3562


for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction6[k])[0],"\n")

<h2> 2.3 OneVsRestClassifier + SGDClassifier with HINGE Loss : </h2>

In [30]:
sgh = SGDClassifier(loss='hinge', class_weight='balanced')

clf = OneVsRestClassifier(sgh)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal',
                                            loss='hinge', max_iter=1000,
                                            n_iter_no_change=5, n_jobs=None,
                                            penalty='l2', power_t=0.5,
                                            random_state=None, shuffle=True,
                                            tol=0.001, validation_fraction=0.1,
                                            verbose=0, warm_start=False),
                    n_jobs=None)

In [31]:
prediction7 = clf.predict(X_test_multilabel)

precision7 = precision_score(y_test_multilabel, prediction7, average='micro')

recall7 = recall_score(y_test_multilabel, prediction7, average='micro')

f1_score7 = 2*((precision7 * recall7)/(precision7 + recall7))

print("precision7: {:.4f}, recall7: {:.4f}, F1-measure: {:.4f}".format(precision7, recall7, f1_score7))

precision7: 0.2711, recall7: 0.3745, F1-measure: 0.3145


for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction7[k])[0],"\n")

<h2> 2.4 OneVsRestClassifier + LogisticRegression:</h2>

In [32]:
lr = LogisticRegression(class_weight='balanced')

clf = OneVsRestClassifier(lr)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight='balanced',
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [33]:
prediction8 = clf.predict(X_test_multilabel)

precision8 = precision_score(y_test_multilabel, prediction8, average='micro')

recall8 = recall_score(y_test_multilabel, prediction8, average='micro')

f1_score8 = 2*((precision8 * recall8)/(precision8 + recall8))

print("precision8: {:.4f}, recall8: {:.4f}, F1-measure: {:.4f}".format(precision8, recall8, f1_score8))

precision8: 0.3248, recall8: 0.4399, F1-measure: 0.3737


for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction8[k])[0],"\n")

<h1>3. TfidfVectorizer with (1 - 3 Grams):<?h1>

In [34]:
start = datetime.now()

#Use tf-idf vectorizer to vectorize the movie topic synopsis
vectorizer = TfidfVectorizer(min_df = 0.000009, max_features =250000,   tokenizer = lambda x: x.split(" "), ngram_range=(1, 3))
X_train_plots_multilabel = vectorizer.fit_transform(X_train_plots)
X_test_plots_multilabel = vectorizer.transform(X_test_plots)

#Use tf-idf vectorizer to vectorize the movie topic synopsis
vectorizer = TfidfVectorizer(min_df = 0.000009, max_features =250000,   tokenizer = lambda x: x.split(" "),  ngram_range=(1, 3))
X_train_topics_multilabel = vectorizer.fit_transform(X_train_topics)
X_test_topics_multilabel = vectorizer.transform(X_test_topics)

#Combine the sparse matrices into a single sparse matrix
from scipy.sparse import hstack
X_train_multilabel=hstack([X_train_plots_multilabel,X_train_topics_multilabel])
X_test_multilabel=hstack([X_test_plots_multilabel,X_test_topics_multilabel])

print("Time taken to run this cell :", datetime.now() - start)

Time taken to run this cell : 0:01:00.376962


In [35]:
print("Dimensions of train data X:",X_train_multilabel.shape, "Y :",y_train_multilabel.shape)
print("Dimensions of test data X:",X_test_multilabel.shape,"Y:",y_test_multilabel.shape)

Dimensions of train data X: (10989, 250247) Y : (10989, 71)
Dimensions of test data X: (2768, 250247) Y: (2768, 71)


<H2> 3.1 OneVsRestClassifier + MultinomialNB :</H2>

In [36]:
mb = MultinomialNB(class_prior = [0.5, 0.5])

clf = OneVsRestClassifier(mb)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=MultinomialNB(alpha=1.0, class_prior=[0.5, 0.5],
                                            fit_prior=True),
                    n_jobs=None)

In [37]:
prediction9 = clf.predict(X_test_multilabel)

precision9 = precision_score(y_test_multilabel, prediction9, average='micro')

recall9 = recall_score(y_test_multilabel, prediction9, average='micro')

f1_score9 = 2*((precision9 * recall9)/(precision9 + recall9))

print("precision9: {:.4f}, recall9: {:.4f}, F1-measure: {:.4f}".format(precision9, recall9, f1_score9))

precision9: 0.6772, recall9: 0.0499, F1-measure: 0.0930


for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction9[k])[0],"\n")

<H2> 3.2 OneVsRestClassifier + SGDClassifier with LOG Loss :</H2>

In [38]:
sgl = SGDClassifier(loss='log', class_weight='balanced')

clf = OneVsRestClassifier(sgl)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal', loss='log',
                                            max_iter=1000, n_iter_no_change=5,
                                            n_jobs=None, penalty='l2',
                                            power_t=0.5, random_state=None,
                                            shuffle=True, tol=0.001,
                                            validation_fraction=0.1, verbose=0,
                                            warm_start=False),
                    n_jobs=None)

In [39]:
prediction10 = clf.predict(X_test_multilabel)

precision10 = precision_score(y_test_multilabel, prediction10, average='micro')

recall10 = recall_score(y_test_multilabel, prediction10, average='micro')

f1_score10 = 2*((precision10 * recall10)/(precision10 + recall10))

print("precision10: {:.4f}, recall10: {:.4f}, F1-measure: {:.4f}".format(precision10, recall10, f1_score10))

precision10: 0.2991, recall10: 0.4454, F1-measure: 0.3578


for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction10[k])[0],"\n")

<h2> 3.3 OneVsRestClassifier + SGDClassifier with HINGE Loss : </h2>

In [40]:
sgh = SGDClassifier(loss='hinge', class_weight='balanced')

clf = OneVsRestClassifier(sgh)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal',
                                            loss='hinge', max_iter=1000,
                                            n_iter_no_change=5, n_jobs=None,
                                            penalty='l2', power_t=0.5,
                                            random_state=None, shuffle=True,
                                            tol=0.001, validation_fraction=0.1,
                                            verbose=0, warm_start=False),
                    n_jobs=None)

In [41]:
prediction11 = clf.predict(X_test_multilabel)

precision11 = precision_score(y_test_multilabel, prediction11, average='micro')

recall11 = recall_score(y_test_multilabel, prediction11, average='micro')

f1_score11 = 2*((precision11 * recall11)/(precision11 + recall11))

print("precision11: {:.4f}, recall11: {:.4f}, F1-measure: {:.4f}".format(precision11, recall11, f1_score11))

precision11: 0.3308, recall11: 0.3639, F1-measure: 0.3465


for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction11[k])[0],"\n")

<h2> 3.4 OneVsRestClassifier + LogisticRegression:</h2>

In [42]:
lr = LogisticRegression(class_weight='balanced')

clf = OneVsRestClassifier(lr)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight='balanced',
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [43]:
prediction12 = clf.predict(X_test_multilabel)

precision12 = precision_score(y_test_multilabel, prediction12, average='micro')

recall12 = recall_score(y_test_multilabel, prediction12, average='micro')

f1_score12 = 2*((precision12 * recall12)/(precision12 + recall12))

print("precision12: {:.4f}, recall12: {:.4f}, F1-measure: {:.4f}".format(precision12, recall12, f1_score12))

precision12: 0.3252, recall12: 0.4399, F1-measure: 0.3739


for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction12[k])[0],"\n")

<h1>4. TfidfVectorizer with (1 - 4 Grams):</h1>

In [44]:
start = datetime.now()

#Use tf-idf vectorizer to vectorize the movie topic synopsis
vectorizer = TfidfVectorizer(min_df = 0.000009, max_features =250000,   tokenizer = lambda x: x.split(" "), ngram_range=(1, 4))
X_train_plots_multilabel = vectorizer.fit_transform(X_train_plots)
X_test_plots_multilabel = vectorizer.transform(X_test_plots)

#Use tf-idf vectorizer to vectorize the movie topic synopsis
vectorizer = TfidfVectorizer(min_df = 0.000009, max_features =250000,   tokenizer = lambda x: x.split(" "),  ngram_range=(1, 4))
X_train_topics_multilabel = vectorizer.fit_transform(X_train_topics)
X_test_topics_multilabel = vectorizer.transform(X_test_topics)

#Combine the sparse matrices into a single sparse matrix
from scipy.sparse import hstack
X_train_multilabel=hstack([X_train_plots_multilabel,X_train_topics_multilabel])
X_test_multilabel=hstack([X_test_plots_multilabel,X_test_topics_multilabel])

print("Time taken to run this cell :", datetime.now() - start)

Time taken to run this cell : 0:01:40.200754


In [45]:
print("Dimensions of train data X:",X_train_multilabel.shape, "Y :",y_train_multilabel.shape)
print("Dimensions of test data X:",X_test_multilabel.shape,"Y:",y_test_multilabel.shape)

Dimensions of train data X: (10989, 250317) Y : (10989, 71)
Dimensions of test data X: (2768, 250317) Y: (2768, 71)


<H2> 4.1 OneVsRestClassifier + MultinomialNB :</H2>

In [46]:
mb = MultinomialNB(class_prior = [0.5, 0.5])

clf = OneVsRestClassifier(mb)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=MultinomialNB(alpha=1.0, class_prior=[0.5, 0.5],
                                            fit_prior=True),
                    n_jobs=None)

In [47]:
prediction13 = clf.predict(X_test_multilabel)

precision13 = precision_score(y_test_multilabel, prediction13, average='micro')

recall13 = recall_score(y_test_multilabel, prediction13, average='micro')

f1_score13 = 2*((precision13 * recall13)/(precision13 + recall13))

print("precision13: {:.4f}, recall13: {:.4f}, F1-measure: {:.4f}".format(precision13, recall13, f1_score13))

precision13: 0.6700, recall13: 0.0545, F1-measure: 0.1008


for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction13[k])[0],"\n")

<h2> 4.2 OneVsRestClassifier + SGDClassifier with LOG Loss :</h2>

In [48]:
sgl = SGDClassifier(loss='log', class_weight='balanced')

clf = OneVsRestClassifier(sgl)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal', loss='log',
                                            max_iter=1000, n_iter_no_change=5,
                                            n_jobs=None, penalty='l2',
                                            power_t=0.5, random_state=None,
                                            shuffle=True, tol=0.001,
                                            validation_fraction=0.1, verbose=0,
                                            warm_start=False),
                    n_jobs=None)

In [49]:
prediction14 = clf.predict(X_test_multilabel)

precision14 = precision_score(y_test_multilabel, prediction14, average='micro')

recall14 = recall_score(y_test_multilabel, prediction14, average='micro')

f1_score14 = 2*((precision14 * recall14)/(precision14 + recall14))

print("precision14: {:.4f}, recall14: {:.4f}, F1-measure: {:.4f}".format(precision14, recall14, f1_score14))

precision14: 0.2944, recall14: 0.4430, F1-measure: 0.3537


for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction14[k])[0],"\n")

<h2> 4.3 OneVsRestClassifier + SGDClassifier with HINGE Loss : </h2>

In [50]:
sgh = SGDClassifier(loss='hinge', class_weight='balanced')

clf = OneVsRestClassifier(sgh)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal',
                                            loss='hinge', max_iter=1000,
                                            n_iter_no_change=5, n_jobs=None,
                                            penalty='l2', power_t=0.5,
                                            random_state=None, shuffle=True,
                                            tol=0.001, validation_fraction=0.1,
                                            verbose=0, warm_start=False),
                    n_jobs=None)

In [51]:
prediction15 = clf.predict(X_test_multilabel)

precision15 = precision_score(y_test_multilabel, prediction15, average='micro')

recall15 = recall_score(y_test_multilabel, prediction15, average='micro')

f1_score15 = 2*((precision15 * recall15)/(precision15 + recall15))

print("precision15: {:.4f}, recall15: {:.4f}, F1-measure: {:.4f}".format(precision15, recall15, f1_score15))

precision15: 0.2981, recall15: 0.3634, F1-measure: 0.3275


for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction15[k])[0],"\n")

<h2> 4.4 OneVsRestClassifier + LogisticRegression:</h2>

In [52]:
lr = LogisticRegression(class_weight='balanced')

clf = OneVsRestClassifier(lr)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight='balanced',
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [53]:
prediction16 = clf.predict(X_test_multilabel)

precision16 = precision_score(y_test_multilabel, prediction16, average='micro')

recall16 = recall_score(y_test_multilabel, prediction16, average='micro')

f1_score16 = 2*((precision16 * recall16)/(precision16 + recall16))

print("precision16: {:.4f}, recall16: {:.4f}, F1-measure: {:.4f}".format(precision16, recall16, f1_score16))

precision16: 0.3251, recall16: 0.4400, F1-measure: 0.3739


for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction16[k])[0],"\n")

<h1>5. TfidfVectorizer with (1 - 5 Grams):</h1>

In [54]:
start = datetime.now()

#Use tf-idf vectorizer to vectorize the movie topic synopsis
vectorizer = TfidfVectorizer(min_df = 0.000009, max_features =250000,   tokenizer = lambda x: x.split(" "), ngram_range=(1, 5))
X_train_plots_multilabel = vectorizer.fit_transform(X_train_plots)
X_test_plots_multilabel = vectorizer.transform(X_test_plots)

#Use tf-idf vectorizer to vectorize the movie topic synopsis
vectorizer = TfidfVectorizer(min_df = 0.000009, max_features =250000,   tokenizer = lambda x: x.split(" "),  ngram_range=(1, 5))
X_train_topics_multilabel = vectorizer.fit_transform(X_train_topics)
X_test_topics_multilabel = vectorizer.transform(X_test_topics)

#Combine the sparse matrices into a single sparse matrix
from scipy.sparse import hstack
X_train_multilabel=hstack([X_train_plots_multilabel,X_train_topics_multilabel])
X_test_multilabel=hstack([X_test_plots_multilabel,X_test_topics_multilabel])

print("Time taken to run this cell :", datetime.now() - start)

Time taken to run this cell : 0:02:15.879856


In [55]:
print("Dimensions of train data X:",X_train_multilabel.shape, "Y :",y_train_multilabel.shape)
print("Dimensions of test data X:",X_test_multilabel.shape,"Y:",y_test_multilabel.shape)

Dimensions of train data X: (10989, 250377) Y : (10989, 71)
Dimensions of test data X: (2768, 250377) Y: (2768, 71)


<H2> 5.1 OneVsRestClassifier + MultinomialNB :</H2>

In [56]:
mb = MultinomialNB(class_prior = [0.5, 0.5])

clf = OneVsRestClassifier(mb)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=MultinomialNB(alpha=1.0, class_prior=[0.5, 0.5],
                                            fit_prior=True),
                    n_jobs=None)

In [57]:
prediction17 = clf.predict(X_test_multilabel)

precision17 = precision_score(y_test_multilabel, prediction17, average='micro')

recall17 = recall_score(y_test_multilabel, prediction17, average='micro')

f1_score17 = 2*((precision17 * recall17)/(precision17 + recall17))

print("precision17: {:.4f}, recall17: {:.4f}, F1-measure: {:.4f}".format(precision17, recall17, f1_score17))

precision17: 0.6586, recall17: 0.0572, F1-measure: 0.1052


for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction17[k])[0],"\n")

<h2> 5.2 OneVsRestClassifier + SGDClassifier with LOG Loss:</h2>

In [58]:
sgl = SGDClassifier(loss='log', class_weight='balanced')

clf = OneVsRestClassifier(sgl)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal', loss='log',
                                            max_iter=1000, n_iter_no_change=5,
                                            n_jobs=None, penalty='l2',
                                            power_t=0.5, random_state=None,
                                            shuffle=True, tol=0.001,
                                            validation_fraction=0.1, verbose=0,
                                            warm_start=False),
                    n_jobs=None)

In [59]:
prediction18 = clf.predict(X_test_multilabel)

precision18 = precision_score(y_test_multilabel, prediction18, average='micro')

recall18 = recall_score(y_test_multilabel, prediction18, average='micro')

f1_score18 = 18*((precision18 * recall18)/(precision18 + recall18))

print("precision18: {:.4f}, recall18: {:.4f}, F1-measure: {:.4f}".format(precision18, recall18, f1_score18))

precision18: 0.2934, recall18: 0.4540, F1-measure: 3.2081


for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction18[k])[0],"\n")

<h2> 5.3 OneVsRestClassifier + SGDClassifier with Hinge Loss:</h2>

In [60]:
sgh = SGDClassifier(loss='hinge', class_weight='balanced')

clf = OneVsRestClassifier(sgh)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal',
                                            loss='hinge', max_iter=1000,
                                            n_iter_no_change=5, n_jobs=None,
                                            penalty='l2', power_t=0.5,
                                            random_state=None, shuffle=True,
                                            tol=0.001, validation_fraction=0.1,
                                            verbose=0, warm_start=False),
                    n_jobs=None)

In [61]:
prediction19 = clf.predict(X_test_multilabel)

precision19 = precision_score(y_test_multilabel, prediction19, average='micro')

recall19 = recall_score(y_test_multilabel, prediction19, average='micro')

f1_score19 = 2*((precision19 * recall19)/(precision19 + recall19))

print("precision19: {:.4f}, recall19: {:.4f}, F1-measure: {:.4f}".format(precision19, recall19, f1_score19))

precision19: 0.2467, recall19: 0.3770, F1-measure: 0.2983


for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction19[k])[0],"\n")

<h2> 5.4  OneVsRestClassifier + LogisticRegression:</h2>

In [62]:
lr = LogisticRegression(class_weight='balanced')

clf = OneVsRestClassifier(lr)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight='balanced',
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [63]:
prediction20 = clf.predict(X_test_multilabel)

precision20 = precision_score(y_test_multilabel, prediction20, average='micro')

recall20 = recall_score(y_test_multilabel, prediction20, average='micro')

f1_score20 = 2*((precision20 * recall20)/(precision20 + recall20))

print("precision20: {:.4f}, recall20: {:.4f}, F1-measure: {:.4f}".format(precision20, recall20, f1_score20))

precision20: 0.3249, recall20: 0.4400, F1-measure: 0.3738


for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction20[k])[0],"\n")

<h1> Conclusion: </h1>

In [64]:
from prettytable import PrettyTable

tabel = PrettyTable()

tabel.field_names=['Model','Vectorizer','ngrams','Precision','recall','f1_score']



tabel.add_row(['MultinomialNB', 'TfidfVectorizer', '(1, 1)', round(precision1, 3),round(recall1, 3), round(f1_score1, 3)])

tabel.add_row(['SGDClassifier(log)', 'TfidfVectorizer', '(1, 1)', round(precision2, 3), round(recall2, 3), round(f1_score2, 3)])

tabel.add_row(['SGDClassifier(hinge)','TfidfVectorizer','(1, 1)' ,round(precision3, 3),  round(recall3, 3), round(f1_score3, 3)])

tabel.add_row(['LogisticRegression','TfidfVectorizer','(1, 1)', round(precision4, 3), round(recall4, 3), round(f1_score4, 3)])

tabel.add_row(['','','','','',''])
tabel.add_row(['','','','','',''])


tabel.add_row(['MultinomialNB', 'TfidfVectorizer', '(1, 2)', round(precision5, 3), round(recall5, 3),  round(f1_score5, 3)])

tabel.add_row(['SGDClassifier(log)', 'TfidfVectorizer', '(1, 2)', round(precision6, 3),  round(recall6, 3),  round(f1_score6, 3)])

tabel.add_row(['SGDClassifier(hinge)','TfidfVectorizer','(1, 2)', round(precision7, 3),  round(recall7, 3),  round(f1_score7, 3)])

tabel.add_row(['LogisticRegression','TfidfVectorizer','(1, 2)', round(precision8, 3),  round(recall8, 3),  round(f1_score8, 3)])

tabel.add_row(['','','','','',''])
tabel.add_row(['','','','','',''])


tabel.add_row(['MultinomialNB', 'TfidfVectorizer', '(1, 3)', round(precision9, 3),  round(recall9, 3), round(f1_score9, 3)])

tabel.add_row(['SGDClassifier(log)', 'TfidfVectorizer', '(1, 3)', round(precision10, 3),  round(recall10, 3), round(f1_score10, 3)])

tabel.add_row(['SGDClassifier(hinge)','TfidfVectorizer','(1, 3)', round(precision11, 3),  round(recall11, 3), round(f1_score11, 3)])

tabel.add_row(['LogisticRegression','TfidfVectorizer','(1, 3)', round(precision12, 3),  round(recall12, 3), round(f1_score12, 3)])

tabel.add_row(['','','','','',''])
tabel.add_row(['','','','','',''])

tabel.add_row(['MultinomialNB', 'TfidfVectorizer', '(1, 4)', round(precision13, 3), round(recall13, 3), round(f1_score13, 3)])

tabel.add_row(['SGDClassifier(log)', 'TfidfVectorizer', '(1, 4)', round(precision14, 3), round(recall14, 3), round(f1_score14, 3)])

tabel.add_row(['SGDClassifier(hinge)','TfidfVectorizer','(1, 4)', round(precision15, 3), round(recall15, 3), round(f1_score15, 3)])

tabel.add_row(['LogisticRegression','TfidfVectorizer','(1, 4)', round(precision16, 3),  round(recall16, 3),  round(f1_score16, 3)])


tabel.add_row(['','','','','',''])
tabel.add_row(['','','','','',''])

tabel.add_row(['MultinomialNB', 'TfidfVectorizer', '(1, 5)', round(precision17, 3), round(recall17, 3), round(f1_score17, 3)])

tabel.add_row(['SGDClassifier(log)', 'TfidfVectorizer', '(1, 5)', round(precision18, 3), round(recall18, 3), round(f1_score18, 3)])

tabel.add_row(['SGDClassifier(hinge)','TfidfVectorizer','(1, 5)', round(precision19, 3), round(recall19, 3), round(f1_score19, 3)])

tabel.add_row(['LogisticRegression','TfidfVectorizer','(1, 5)', round(precision20, 3),  round(recall20, 3),  round(f1_score20, 3)])

print(tabel)

+----------------------+-----------------+--------+-----------+--------+----------+
|        Model         |    Vectorizer   | ngrams | Precision | recall | f1_score |
+----------------------+-----------------+--------+-----------+--------+----------+
|    MultinomialNB     | TfidfVectorizer | (1, 1) |   0.658   | 0.054  |  0.099   |
|  SGDClassifier(log)  | TfidfVectorizer | (1, 1) |   0.273   | 0.457  |  0.342   |
| SGDClassifier(hinge) | TfidfVectorizer | (1, 1) |    0.3    | 0.383  |  0.336   |
|  LogisticRegression  | TfidfVectorizer | (1, 1) |   0.302   | 0.445  |   0.36   |
|                      |                 |        |           |        |          |
|                      |                 |        |           |        |          |
|    MultinomialNB     | TfidfVectorizer | (1, 2) |   0.679   | 0.044  |  0.083   |
|  SGDClassifier(log)  | TfidfVectorizer | (1, 2) |   0.295   | 0.449  |  0.356   |
| SGDClassifier(hinge) | TfidfVectorizer | (1, 2) |   0.271   | 0.374  |  0.

# For Topic Modelling, I had tried many variations but  the maximum f1 score we are getting is 0.374 for LogisticRegression But in a particular case that is SGDClassifier with (1, 5) grams we got the f1 score of 3.208